In [1]:
import os
import pandas as pd
from numpy import sqrt
from itertools import combinations
from scipy.stats import chi2, friedmanchisquare, norm, wilcoxon

#### Wilcoxon $Z$-score (for large samples)

$$
z^{*} = \displaystyle\frac{T - \bar{x}_T}{s_T}
$$

where $\bar{x}_T = \displaystyle\frac{n(n+1)}{4}$ and $s_T = \sqrt{\displaystyle\frac{n(n+1)(2n+1)}{24}}$.

In [2]:
def wilcoxon_z_score(sample_size, ranks_sum):
    x_bar = sample_size * (sample_size + 1)/4
    std_dev = sqrt((sample_size * (sample_size + 1) * (2*sample_size + 1))/24.)
    return (ranks_sum - x_bar)/std_dev

***

In [3]:
results_folder = '../results/'
simulations = ['C', 'T1', 'T2']
variables = ['alpha', 'x_min', 'gini', 'mean', 'btm', 'top', 'gdp']

In [4]:
filenames = [filename for filename in os.listdir(results_folder) if filename.endswith('.csv')]

In [5]:
dfs = {sim: pd.read_csv(results_folder + filename, index_col='time')
       for sim, filename in zip(simulations, filenames)}

***

## Friedman's tests:

Non-parametric test to asses whether there are statistically significant differences among the simulations' results.

Hypotheses:

$$
\begin{align}
& H_0 : \theta^{C}_{s} = \theta^{T1}_{s} = \theta^{T2}_{s} \\
& H_1 : \theta^{C}_{s} \neq \theta^{T1}_{s} \vee \theta^{C}_{s} = \theta^{T2}_{s} \vee \theta^{T1}_{s} = \theta^{T2}_{s}
\end{align}
$$

In [6]:
alpha = 0.01
df = len(simulations) - 1

for variable in variables:
    print(f'\nVARIABLE {variable}')
    stat, pval = friedmanchisquare(dfs['C'][variable], dfs['T1'][variable], dfs['T2'][variable])
    crit = chi2.ppf(1-alpha, df)
    print(f'\n\tTest statistic: {stat:.1f}')
    print(f'\n\tCritical value: {crit:.1f}')
    print(f'\n\tP-value.......: {pval:.2e}')
    
    result = 'H0 REJECTED.' if pval < alpha else 'Failed to reject H0.'
    print(f'\n\tResult........: {result}')


VARIABLE alpha

	Test statistic: 100.0

	Critical value: 9.2

	P-value.......: 1.93e-22

	Result........: H0 REJECTED.

VARIABLE x_min

	Test statistic: 100.0

	Critical value: 9.2

	P-value.......: 1.93e-22

	Result........: H0 REJECTED.

VARIABLE gini

	Test statistic: 100.0

	Critical value: 9.2

	P-value.......: 1.93e-22

	Result........: H0 REJECTED.

VARIABLE mean

	Test statistic: 79.0

	Critical value: 9.2

	P-value.......: 7.00e-18

	Result........: H0 REJECTED.

VARIABLE btm

	Test statistic: 100.0

	Critical value: 9.2

	P-value.......: 1.93e-22

	Result........: H0 REJECTED.

VARIABLE top

	Test statistic: 96.2

	Critical value: 9.2

	P-value.......: 1.32e-21

	Result........: H0 REJECTED.

VARIABLE gdp

	Test statistic: 80.8

	Critical value: 9.2

	P-value.......: 2.91e-18

	Result........: H0 REJECTED.


***

## Wilcoxon signed rank tests

*Ad hoc* test to find between which pairs of simulations there is statistically significant difference.

Hypotheses:

$$
\begin{align}
& H_0 : \mu_D = 0\\
& H_1 : \mu_D \neq 0
\end{align}
$$

In [7]:
alpha_B = alpha/len(simulations)

for variable in variables:
    print(f'\nVARIABLE {variable}')

    for s1, s2 in combinations(simulations, 2):
        print(f'\n  Test: {s1} x {s2}')
        ranks_sum, pval = wilcoxon(dfs[s1][variable], dfs[s2][variable], zero_method='pratt')
        z_score = wilcoxon_z_score(len(dfs[s1][variable]), ranks_sum)
        z_crit = norm.ppf(1-alpha_B/2.)
        print(f'\n\tRanks sum.: {ranks_sum:.1f}')
        print(f'\n\tZ-score...: {z_score:.3f}')
        print(f'\n\tZ critical: {z_crit:.3f}')
        print(f'\n\tP-value...: {pval:.2e}')
        
        result = 'H0 REJECTED.' if abs(z_score) > z_crit else 'Failed to reject H0.'
        print(f'\n\tResult........: {result}')


VARIABLE alpha

  Test: C x T1

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

  Test: C x T2

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

  Test: T1 x T2

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

VARIABLE x_min

  Test: C x T1

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

  Test: C x T2

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

  Test: T1 x T2

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

VARIABLE gini

  Test: C x T1

	Ranks sum.: 0.0

	Z-score...: -6.154

	Z critical: 2.935

	P-value...: 7.56e-10

	Result........: H0 REJECTED.

  Test: C x T2

	Ranks sum.: 0.0

	Z-score...: 